In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Activation, Dropout, Flatten, Concatenate
from keras.layers import Conv2D, Add, Reshape
import tensorflow as tf
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('checkpoint.h5', monitor='val_accuracy',save_best_only=True,verbose=2)
layer1 = Conv2D(filters=1, kernel_size=[1,1], strides=(1,1), padding='valid', name='Utilities',kernel_constraint=Between(-1,-1),use_bias=False, trainable=True)
layer2 = Conv2D(filters=1, kernel_size=[1,1], strides=(1,1), padding='valid', name='Utilitiess',use_bias=False, trainable=True)
layer3 = Conv2D(filters=1, kernel_size=[1,1], strides=(1,1), padding='valid', name='UtilitiesSP',use_bias=False, trainable=True) 
#MNL utilities
main_input= Input((1,2,1), name = 'Features')
utilities = layer1 (main_input)
utilitiesR = Reshape([2], name='Flatten_Dim')(utilities)
#MNL utilities1
main_input1= Input((1,2,1), name = 'Featuress')
utilitiess = layer2 (main_input1)
utilitiesRR= Reshape([2], name='Flatten_Dimm')(utilitiess)
#SPvariables
SP_input= Input((1,2,1), name = 'Features2')
utilities2 = layer1 (SP_input)
utilities3 = layer3 (utilities2)
utilitiesR2 = Reshape([2], name='Flatten_Dim2')(utilities3)
#SPvariables1
SP_input1= Input((1,2,1), name = 'Featuress2')
utilitiess2 = layer2 (SP_input1)
utilitiess3 = layer3 (utilitiess2)
utilitiesRR2 = Reshape([2], name='Flatten_Dimm2')(utilitiess3)
#extra variables
extra_input = Input((4,1,1))
extra_inputt1 = Reshape([1,4],name='Extra_Input1')(extra_input)
hidden_layers = 2
x = extra_inputt1 
for i in range(hidden_layers-1):
    x = Dense(units=1, activation='sigmoid', name="Dense{}".format(i))(x)
    x = Dropout(0.2, name='Drop{}'.format(i))(x)
dropped = x
new_feature = Dense(units=2, name="Output_new_feature")(dropped)
new_featureR = Reshape([2], name='Remove_Dim')(new_feature)
#adding four components of utilities
final_utilities = Add(name="New_Utility_functions")([utilitiesR,utilitiesRR, utilitiesR2, utilitiesRR2, new_featureR])
logits = Activation('softmax', name='Choice')(final_utilities)
model = Model(inputs=[main_input,main_input1, SP_input,SP_input1, extra_input], outputs=logits)
model.summary()

In [ ]:
import numpy as np
import tensorflow as tf
import random
import shelve
import _pickle as pickle
from keras.models import Sequential, Model
from keras.callbacks import TensorBoard, EarlyStopping
from keras.models import load_model
from keras.layers import Input, Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, Add, Reshape
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils
from keras.losses import mean_squared_error
from keras.callbacks import EarlyStopping
#train_utilities
train_data = np.load('/home/rahul/interaction2/TCinteractionwillingnessspace/1/MNLneural_result.npy')
train_labels = train_data[:,-1,:]
train_data = np.delete(train_data, -1, axis = 1)
#train_utilities1
train_data1 = np.load('/home/rahul/interaction2/TCinteractionwillingnessspace/1/MNLneural_result1.npy')
#SP utilities
SP_data = np.load('/home/rahul/interaction2/TCinteractionwillingnessspace/1/MNLneural_SP.npy')
#SP utilities1
SP_data1 = np.load('/home/rahul/interaction2/TCinteractionwillingnessspace/1/MNLneural_SP1.npy')
#extradata in utilities
extra_data = np.load('/home/rahul/interaction2/TCinteractionwillingnessspace/1/MNLneural_extra.npy')
extra_data  = np.delete(extra_data, -1, axis = 2)
def normalize(data):
    return (data-data.mean(axis=0))/(data.std(axis=0))
extra_data = normalize(extra_data)
#estimation
#callback = EarlyStopping(monitor ='loss', min_delta=0.0004, patience =5,restore_best_weights=True)
model.compile(optimizer = Adam(clipnorm = 50.), metrics = [tf.keras.metrics.CategoricalAccuracy()], loss = 'categorical_crossentropy')
history = model.fit([train_data1,train_data, SP_data1,SP_data,extra_data],train_labels, epochs = 2000, steps_per_epoch = 50, verbose = 2,callbacks =[model_checkpoint])
betas_layer = model.get_layer(name = 'Utilities')
betas_layer1 = model.get_layer(name = 'Utilitiess')
betas_layer3 = model.get_layer(name = 'UtilitiesSP')
betas = betas_layer.get_weights()
betas1 = betas_layer1.get_weights()
betas3 = betas_layer3.get_weights()
evaluation = model.evaluate([train_data1,train_data, SP_data1,SP_data,extra_data],train_labels,verbose=2)